### IMPORT NECESSARY LIBRARIES

In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import openpyxl
import sqlite3
import time
from bs4 import BeautifulSoup
import requests

### CREATE AN EMPTY EXCEL SHEET TO TAKE THE DATA TO BE SCRAPED

In [ ]:
# create an excel file
IMDB_Movies_Excel = openpyxl.Workbook()

# Assign the excel sheet and name it
sheet = IMDB_Movies_Excel.active
sheet.title = "IMDB Movies Rating"

sheet.append(['Rank', 'Name', 'Year', 'IMDB_Rating'])

### WEBSCRAPING, CONVERTING SCRAPED DATA TO A LIST

### DATABASE SETUP

In [4]:
# Database setup
conn = sqlite3.connect('recipes.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS RECIPES (title TEXT, ingredients TEXT, rating REAL)''')

In [5]:
# Define parameters
target_site = 'https://www.allrecipes.com/recipes/'

In [3]:
# Main function to scrape the main page and individual recipe pages
def main():
    main_url = 'https://www.allrecipes.com/recipes/'
    response = requests.get(main_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the recipe links (modify the selector as per your requirement)
    links = soup.find_all('a', href=True)

    # Extract the href attributes to get the URLs
    urls = [link['href'] for link in links if '/recipe/' in link['href']]

    # Iterate over the URLs and scrape data
    for url in urls:
        try:
            title, ingredients, rating = scrape_recipe(url)
            print(f'Scraped {title}')

            # Insert the data into the database
            c.execute('INSERT INTO recipes (title, ingredients, rating) VALUES (?, ?, ?)',
                      (title, ingredients, rating))
            conn.commit()
        except Exception as e:
            print(f'Failed to scrape {url}: {e}')